In [13]:
from llama_index.llms.openai_like import OpenAILike

llm = OpenAILike(
    model="Qwen/Qwen2.5-32B-Instruct",
    api_base="http://dgx3.humboldt.edu:8000/v1",
    context_window=128000,
    api_key="2e7f622f5a3c76b9f718205ddd0b860c8b7c6d4fb883b436797a91ee3e0cab3d",
    is_chat_model=True,
    is_function_calling_model=True,
)

In [14]:
import nest_asyncio

nest_asyncio.apply()

In [15]:
from llama_index.core.llms import ChatMessage
llm.chat(messages=[ChatMessage(role="user", content="Hi")])

ChatResponse(message=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='Hello! How can I assist you today?')]), raw=ChatCompletion(id='chatcmpl-JqqLm1DVmed5LEKU04YuxISSb2MS361e', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1744522608, model='Qwen/Qwen2.5-32B-Instruct', object='chat.completion', service_tier=None, system_fingerprint='b5121-c94085df', usage=CompletionUsage(completion_tokens=10, prompt_tokens=30, total_tokens=40, completion_tokens_details=None, prompt_tokens_details=None), timings={'prompt_n': 25, 'prompt_ms': 159.516, 'prompt_per_token_ms': 6.38064, 'prompt_per_second': 156.72409037337948, 'predicted_n': 10, 'predicted_ms': 622.519, 'predicted_per_token_ms': 62.2519, 'predicted_per_second': 16.063766728405074}), delta=No

In [16]:
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore

# Note: used to be `Neo4jPGStore`
graph_store = Neo4jPropertyGraphStore(
    username="neo4j",
    password="llamaindex",
    url="bolt://localhost:7687",
)

In [17]:
import json
import psycopg
import ray.data
import os
from llama_index.core import Document
from dotenv import load_dotenv
load_dotenv()

conn_str = os.getenv("PG_CONN_STR")

ds = ray.data.read_sql("SELECT * FROM documents", lambda: psycopg.connect(conn_str))

documents = ds.select_columns(["document"])

documents = documents.take_all()

documents = [Document.from_dict(json.loads(d['document'])) for d in documents]

documents = documents[:50]

for document in documents:
    document.excluded_llm_metadata_keys = ["abstract", "url"]
    document.excluded_embed_metadata_keys = [
        "abstract",
        "url",
        "download_link",
        "publication_date",
    ]


    # Filter discipline list
    discipline = document.metadata.get("discipline")
    if discipline is not None:
        discipline = list(set(discipline))  # Only unique values
        document.metadata["discipline"] = discipline

    # Limit number of authors shown in document
    author = document.metadata.get("author")
    if author is not None:
        author = list(set(author))
        document.metadata["author"] = author



Python-dotenv could not parse statement starting at line 15
Python-dotenv could not parse statement starting at line 18
2025-04-13 05:36:55,891	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2025-04-11_18-21-54_817665_2756939/logs/ray-data
2025-04-13 05:36:55,893	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadSQL] -> TaskPoolMapOperator[Project]
Running 0: 0.00 row [00:00, ? row/s]
Running Dataset. Active & requested resources: 1/80 CPU, 333.1MB/74.0GB object store:  13%|█▎        | 825/6.50k [00:01<00:07, 808 row/s]



Running Dataset. Active & requested resources: 5/80 CPU, 340.6MB/74.0GB object store:  53%|█████▎    | 3.45k/6.50k [00:02<00:01, 1.86k row/s]



                                                                                                                                             
                                                                       

In [18]:
from llama_index.core import PropertyGraphIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor

index = PropertyGraphIndex.from_documents(
    documents,
    embed_model=HuggingFaceEmbedding(
        model_name="sentence-transformers/all-mpnet-base-v2"
    ),
    kg_extractors=[
        SchemaLLMPathExtractor(
            llm=llm
        )
    ],
    property_graph_store=graph_store,
    show_progress=True,
)

Parsing nodes: 100%|██████████| 50/50 [00:00<00:00, 209.80it/s]


APITimeoutError: Request timed out.

Retrying llama_index.llms.openai.base.OpenAI._achat in 1.0 seconds as it raised APIConnectionError: Connection error..
Retrying llama_index.llms.openai.base.OpenAI._achat in 1.0 seconds as it raised APIConnectionError: Connection error..
Retrying llama_index.llms.openai.base.OpenAI._achat in 1.0 seconds as it raised APIConnectionError: Connection error..
Retrying llama_index.llms.openai.base.OpenAI._achat in 1.0 seconds as it raised APIConnectionError: Connection error..
Retrying llama_index.llms.openai.base.OpenAI._achat in 1.8867944117578586 seconds as it raised APIConnectionError: Connection error..
Retrying llama_index.llms.openai.base.OpenAI._achat in 1.83584521926418 seconds as it raised APIConnectionError: Connection error..
Retrying llama_index.llms.openai.base.OpenAI._achat in 1.3843145349460317 seconds as it raised APIConnectionError: Connection error..
Retrying llama_index.llms.openai.base.OpenAI._achat in 1.9007317870150806 seconds as it raised APIConnectionError: Connectio